In [17]:
import numpy as np
import tensorflow as tf
import os

In [18]:
TFRECORDS_DIR = '../data/lsms_tfrecords/'

In [19]:
BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'avg_rad']

In [20]:
def create_single_feature_set(filename):
    record = tf.data.TFRecordDataset(filenames=[filename])
    feature_set = parse_features(record=record)
    
    return feature_set

def parse_features(record):
    raw_example = next(iter(record)) 
    example = tf.train.Example.FromString(raw_example.numpy())
    
    return example.features.feature

In [57]:
tfrecords = sorted([f for f in os.listdir(TFRECORDS_DIR) if not f.startswith('.')])
for i, file in enumerate(tfrecords):
    tfrecords[i] = os.path.join('../data/lsms_tfrecords', file)
num_records = len(tfrecords)

def calculate_band_means():
    band_means = {'BLUE': 0, 'GREEN': 0, 'RED': 0, 'NIR': 0, 'SW_IR1': 0, 'SW_IR2': 0, 'TEMP': 0, 'avg_rad': 0}

    for tf_r in tfrecords:
        feature_set = create_single_feature_set(filename=tf_r)

        for band_name in BANDS:
            band = np.array(feature_set[band_name].float_list.value, dtype=np.float32).reshape(255, 255)
            band_means[band_name] += np.mean(band[:,:])
            
    band_means = {key: value / num_records for key, value in band_means.items()}
    
    return band_means        

In [ ]:
band_means = calculate_band_means()